# Applied Data Science Specialisation Capstone

##### This notebook is for the Applied Data Science Capstone course project . ( Part of the IBM Data Science Professional Certification )

## The Battle of Neighbourhoods - Neighbourhoods Segmentation and Clustering

In this project, given a city like the City of Toronto, we will segment it into different neighbourhoods using the geographical coordinates of the centre of each neighbourhood. Then, using a combination of location data and machine learning, we will group the neighbourhoods into clusters.

In [2]:
# import required libraries
import pandas as pd
import numpy as np

In [6]:
# test cell
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
